## FSL FEAT in nipype

In [2]:
import os
import nipype
import nipype.interfaces.io as nio
import nipype.interfaces.fsl as fsl
import nipype.interfaces.ants as ants
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.algorithms.modelgen as model

import glob

240913-13:58:14,195 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.1


## Set-up preprocessing

basically we only need SUSAN smoothing

In [5]:
# general set-up
project_folder = '/home/Public/trondheim'
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')

smoothing_fwhm = 1.5#4.5 # 4.5 # or 1.5
t_r = 1.38
hpcutoff = 128.   # seconds

# task-specific part
subject_ids = [str(x).zfill(3) for x in range(2, 47)]
sessions = ['grids']
tasks = ['grids']
# sessions = ['rlsat']
# tasks = ['rlsat']
spaces = ['T1w'] #, 'MNI152NLin2009cAsym']

smoothing_str = str(smoothing_fwhm).replace('.', 'p')
## check for subject ids to run
# subject_ids = [x for x in subject_ids if not os.path.exists(f'../derivatives/susan_smoothed_func/sub-{x}/ses-rbrevl/func/sub-{x}_ses-rbrevl_task-rb_run-1_space-T1w_desc-preproc_bold_smoothed_fwhm-{smoothing_str}.nii.gz')]

# subs_ = [s.split('-')[1] for s in [x.split('/')[4] for x in sorted(glob.glob('../derivatives/fmriprep/fmriprep/sub-*/ses-sstmsit'))]]
#subs_ = [s.split('-')[1] for s in [x.split('/')[4] for x in sorted(glob.glob('../derivatives/fmriprep/fmriprep/sub-*/ses-rlsat'))]]
subs_ = [s.split('-')[1] for s in [x.split('/')[4] for x in sorted(glob.glob('../derivatives/fmriprep/fmriprep/sub-*/ses-grids'))]]
# already_run = [s.split('-')[1] for s in [x.split('/')[3]for x in sorted(glob.glob('../derivatives/susan_smoothed_hp_func/sub-*/ses-sstmsit/func/*task*1p5*'))]]
# to_run = [x for x in subs_ if x not in already_run]
# subject_ids = subs_
subject_ids = ['060']
# subject_ids

# subject_ids = [x for x in subject_ids if not os.path.exists(f'../derivatives/susan_smoothed_func/sub-{x}/ses-sstmsit/func/sub-{x}_ses-sstmsit_task-sst_run-1_space-T1w_desc-preproc_bold_smoothed_fwhm-{smoothing_str}.nii.gz')]
#subject_ids = [x for x in subject_ids if not os.path.exists(f'../derivatives/susan_smoothed_func/sub-{x}/ses-rlsat/func/sub-{x}_ses-rlsat_task-rlsat_run-1_space-T1w_desc-preproc_bold_smoothed_fwhm-{smoothing_str}.nii.gz')]
# subject_ids = subject_ids = [x for x in subject_ids if not os.path.exists(f'../derivatives/susan_smoothed_func/sub-{x}/ses-mrlc/func/sub-{x}_ses-mrlc_task-mt_run-1_space-T1w_desc-preproc_bold_smoothed_fwhm-{smoothing_str}.nii.gz')]
subject_ids

['060']

In [8]:
# subject_ids = [x.split('/')[3].split('-')[1] for x in sorted(glob.glob('../sourcedata/zipdata/sub-*/ses-sstmsit'))]
# subject_ids = [s for s in subject_ids if not os.path.exists(f'../derivatives/susan_smoothed_func/sub-{s}/ses-sstmsit/func/sub-{s}_ses-sstmsit_task-sst_run-1_space-T1w_desc-preproc_bold_smoothed_fwhm-{smoothing_str}.nii.gz')]
# subject_ids

In [15]:
# subject_ids = ['002','003','004','005','006','007','008','009','010','011', '012','013','014',
#                '015','016','017','018','019','020','023','024','025','026', '027', 
#                '028', '029','031', '032', '033', '034', '035', '037', '038', '039'
#                '041', '042', '043', '044']
# subject_ids = ['027','029','030','031','032']
# subject_ids=['044']

subject_ids.remove('053') # no grids session in /home/Public/trondheim/derivatives/fmriprep/fmriprep/sub-*/
subject_ids.remove('054') # no grids session
subject_ids.remove('058') # no grids session
subject_ids.remove('082') # no grids session
subject_ids

['045',
 '046',
 '048',
 '050',
 '051',
 '057',
 '059',
 '064',
 '065',
 '066',
 '068',
 '069',
 '070',
 '071',
 '073',
 '074',
 '075',
 '076',
 '078',
 '080',
 '083',
 '084',
 '085',
 '086',
 '087',
 '088']

## Preprocess (i.e. smooth with SUSAN)

In [6]:
workflow = pe.Workflow(name='smooth_wf2')
workflow.base_dir = os.path.join(work_dir, 'smoothing_wf2')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# subjects & identity1
identity = pe.Node(util.IdentityInterface(fields=['subject_id', 'ses', 'task', 'space']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('ses', sessions),
                      ('task', tasks),
                      ('space', spaces)]

# file selector
templates = {'func': os.path.join(project_folder, 'derivatives', 'fmriprep', 'fmriprep', 
                                  'sub-{subject_id}', 'ses-{ses}', 'func', 
                                  'sub-{subject_id}_ses-{ses}_task-{task}_run-*_space-{space}_desc-preproc_bold.nii.gz'),
             'mask': os.path.join(project_folder, 'derivatives', 'fmriprep', 'fmriprep', 
                                  'sub-{subject_id}', 'ses-{ses}', 'func', 
                                  'sub-{subject_id}_ses-{ses}_task-{task}_run-*_space-{space}_desc-brain_mask.nii.gz'),
             }
selector = pe.Node(nio.SelectFiles(templates), name='selector')

#
workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'ses', selector, 'ses')
workflow.connect(identity, 'task', selector, 'task')
workflow.connect(identity, 'space', selector, 'space')

# convert to float
prefiltered_func_data = pe.MapNode(interface=fsl.ImageMaths(out_data_type='float',
                                             op_string = '',
                                             suffix='_dtype'),
                       iterfield=['in_file'],
                       name='convert2float', mem_gb=6)   # about 6GB in T1w space

workflow.connect(selector, 'func', prefiltered_func_data, 'in_file')

# Determine the 2nd and 98th percentile intensities of each functional 



# getthresh = pe.MapNode(interface=fsl.ImageStats(op_string='-p 2 -p 98'),
#                        iterfield = ['in_file'],
#                        name='getthreshold')

# workflow.connect(prefiltered_func_data, 'out_file', getthresh, 'in_file')

# Threshold the first run of the functional data at 10% of the 98th percentile
# threshold = pe.MapNode(interface=fsl.ImageMaths(out_data_type='char',
#                                              suffix='_thresh'),
#                     iterfield = ['in_file'],
#                     name='threshold')

# # Define a function to get 10% of the intensity
# def getthreshop(thresh):
#     return '-thr %.10f -Tmin -bin'%(0.1*thresh[0][1])

# workflow.connect(prefiltered_func_data, 'out_file', threshold, 'in_file')
# workflow.connect(getthresh, ('out_stat', getthreshop), threshold, 'op_string')

# Determine the median value of the functional runs using the mask
medianval = pe.MapNode(interface=fsl.ImageStats(op_string='-k %s -p 50'),
                       iterfield = ['in_file','mask_file'],
                       name='medianval')

workflow.connect(prefiltered_func_data, 'out_file', medianval, 'in_file')
workflow.connect(selector, 'mask', medianval, 'mask_file')


# # Dilate the mask. This is the final mask for level 1.
# dilatemask = pe.MapNode(interface=fsl.ImageMaths(suffix='_dil',
#                                               op_string='-dilF'),
#                     iterfield=['in_file'],
#                      name='dilatemask')

# workflow.connect(threshold, 'out_file', dilatemask, 'in_file')


# Mask the motion corrected functional runs with the dilated mask
prefiltered_func_data_thresh = pe.MapNode(interface=fsl.ImageMaths(suffix='_mask',
                                                op_string='-mas'),
                       iterfield=['in_file','in_file2'],
                       name='apply_brain_mask')

workflow.connect(prefiltered_func_data, 'out_file', prefiltered_func_data_thresh, 'in_file')
workflow.connect(selector, 'mask', prefiltered_func_data_thresh, 'in_file2')


# Determine the mean image from each functional run
meanfunc2 = pe.MapNode(interface=fsl.ImageMaths(op_string='-Tmean',
                                                suffix='_mean'),
                       iterfield=['in_file'],
                       name='meanfunc2')

workflow.connect(prefiltered_func_data_thresh, 'out_file', meanfunc2, 'in_file')


# Merge the median values with the mean functional images into a coupled list
# #Yes, it is Node with iterfield! Not MapNode.
mergenode = pe.Node(interface=util.Merge(2, axis='hstack'),
                       iterfield=['in1','in2'],
                       name='merge')

workflow.connect(meanfunc2,'out_file', mergenode, 'in1')
workflow.connect(medianval,'out_stat', mergenode, 'in2')


# Smooth each run using SUSAN with the brightness threshold set to 75% of the median value for each run 
# and a mask constituting the mean functional
smooth = pe.MapNode(interface=fsl.SUSAN(),
                    iterfield=['in_file', 'brightness_threshold', 'usans'],
                    name='smooth', mem_gb=10)  # reserve up to 10 GB for this node (T1w space only! otherwise use 25)
smooth.inputs.fwhm = smoothing_fwhm


# get brightness thresholds for SUSAN
def getbtthresh(medianvals):
    return [0.75*val for val in medianvals]

def getusans(x):
    ## return the mean, and 0.75* the median of the func run
    return [[tuple([val[0],0.75*val[1]])] for val in x]

workflow.connect(prefiltered_func_data_thresh, 'out_file', smooth, 'in_file')
workflow.connect(medianval, ('out_stat', getbtthresh), smooth, 'brightness_threshold')
workflow.connect(mergenode, ('out', getusans), smooth, 'usans')


# Mask the smoothed data with the dilated mask
maskfunc3 = pe.MapNode(interface=fsl.ImageMaths(suffix='_mask',
                                                op_string='-mas'),
                       iterfield=['in_file', 'in_file2'],
                       name='maskfunc3', mem_gb=10)

workflow.connect(smooth, 'smoothed_file', maskfunc3, 'in_file')
workflow.connect(selector, 'mask', maskfunc3, 'in_file2')


# Scale each volume of the run so that the median value of the run is set to 10000 (FSL convention)
intnorm = pe.MapNode(interface=fsl.ImageMaths(suffix='_intnorm'),
                     iterfield=['in_file','op_string'],
                     name='intnorm', mem_gb=10)

# Define a function to get the scaling factor for intensity normalization
def getinormscale(medianvals):
    return ['-mul %.10f'%(10000./val) for val in medianvals]

workflow.connect(maskfunc3, 'out_file', intnorm, 'in_file')
workflow.connect(medianval, ('out_stat', getinormscale), intnorm, 'op_string')


# datasink
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(project_folder, 'derivatives')
substitutions = [('_ses_%s_space_%s_subject_id_%s_task_%s' % (ses, space, sub, task), 'sub-%s/ses-%s/func/' % (sub, ses))
                   for space in spaces 
                   for ses in sessions 
                   for sub in subject_ids
                   for task in tasks]
substitutions += [('dtype_mask_smooth_mask_intnorm', 'smoothed_fwhm-' + str(smoothing_fwhm).replace('.', 'p'))]
substitutions += [('_intnorm%d'%run, '') for run in [0,1,2,3]]  # get rid of subfolder per run
substitutions += [('_highpass%d'%run, '') for run in [0,1,2,3]] 
ds.inputs.substitutions = substitutions

workflow.connect(intnorm, 'out_file', ds, 'susan_smoothed_func')  ## smoothed functional data


# Highpass filtering below - we *do* want to do this - or maybe not?
# Create tempMean
tempMean = pe.MapNode(interface=fsl.ImageMaths(op_string='-Tmean',
                                                suffix='_mean'),
                       iterfield=['in_file'],
                       name='tempMean')

workflow.connect(intnorm, 'out_file', tempMean, 'in_file')


# Perform temporal highpass filtering on the data. This is the same as filtered_func_data in FSL output.
highpass = pe.MapNode(interface=fsl.ImageMaths(op_string= '-bptf %d -1 -add'%(hpcutoff/(2*t_r)), suffix='_tempfilt'),
                      iterfield=['in_file','in_file2'],
                      name='highpass', mem_gb=20)

workflow.connect(tempMean, 'out_file', highpass, 'in_file2')
workflow.connect(intnorm, 'out_file', highpass, 'in_file')

workflow.connect(highpass, 'out_file', ds, 'susan_smoothed_hp_func')
# workflow.connect(maskfunc3, 'out_file', ds, 'masked_func_data')  # don't save, pointless

In [7]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 16, 'memory_gb': 150})

240913-13:58:52,809 nipype.workflow INFO:
	 Workflow smooth_wf2 settings: ['check', 'execution', 'logging', 'monitoring']
240913-13:58:52,961 nipype.workflow INFO:
	 Running in parallel.
240913-13:58:53,169 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 150.00/150.00, Free processors: 16/16.
240913-13:58:53,288 nipype.workflow INFO:
	 [Node] Setting-up "smooth_wf2.selector" in "/home/Public/trondheim/processing/nipype_workflow_folders/smoothing_wf2/smooth_wf2/_ses_grids_space_T1w_subject_id_060_task_grids/selector".
240913-13:58:53,320 nipype.workflow INFO:
	 [Node] Executing "selector" <nipype.interfaces.io.SelectFiles>
240913-13:58:53,326 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.001382s.
240913-13:58:55,144 nipype.workflow INFO:
	 [Job 0] Completed (smooth_wf2.selector).
240913-13:58:55,146 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 150.00/150.00, Free processors: 16/16.


In [5]:
# from nipype.utils.filemanip import loadpkl
# res = loadpkl('/home/Public/trondheim/processing/crashdumps/crash-20220116-122933-scotti-selector.a20-9e5d7a36-8777-4dd2-b1ee-263ec58edb60.pklz')
# res


{'node': smooth_wf2.selector.a20,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 516, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 635, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 741, in _run_command\n    result = self._interface.run(cwd=outdir)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 436, in run\n    outputs = self.aggregate_outputs(runtime)\n',
  '  File "/home/scotti/.co